In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-07-01 17:39:22.081469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Set the path to your training data folder
train_data_dir = 'training'

# Set the number of classes/categories
num_classes = 5

# Set the input image dimensions
img_width, img_height = 224, 224

# Set the batch size for training
batch_size = 32

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,       # Rescale pixel values to [0, 1]
    shear_range=0.2,         # Shear transformations
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True    # Random horizontal flips
)

# Create a generator for training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 245 images belonging to 5 classes.


In [3]:
# Build the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=8
)


2023-07-01 17:39:25.749383: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/8
7/7 [==============================] - 4s 467ms/step - loss: 36.8640 - accuracy: 0.2300
Epoch 2/8
7/7 [==============================] - 3s 460ms/step - loss: 11.9505 - accuracy: 0.3005
Epoch 3/8
7/7 [==============================] - 3s 449ms/step - loss: 2.3518 - accuracy: 0.4821
Epoch 4/8
7/7 [==============================] - 3s 419ms/step - loss: 1.4033 - accuracy: 0.6150
Epoch 5/8
7/7 [==============================] - 3s 453ms/step - loss: 0.6002 - accuracy: 0.7840
Epoch 6/8
7/7 [==============================] - 3s 415ms/step - loss: 0.4238 - accuracy: 0.8451
Epoch 7/8
7/7 [==============================] - 3s 407ms/step - loss: 0.3037 - accuracy: 0.9061
Epoch 8/8
7/7 [==============================] - 3s 421ms/step - loss: 0.2562 - accuracy: 0.9202


In [20]:
# Create a mapping from class index to class label
class_mapping = train_generator.class_indices
class_mapping = {v: k for k, v in class_mapping.items()}
class_mapping

{0: 'Bags',
 1: 'Female TShirts',
 2: 'Male TShirts',
 3: 'Shoes',
 4: 'Tracks and Trousers'}

In [22]:
loaded_model = tf.keras.models.load_model('image_classification_model.h5')


# # Set the path to your single test image
test_image_path = 'pent.jpg'

# Load and preprocess the test image
test_image = tf.keras.preprocessing.image.load_img(
    test_image_path,
    target_size=(img_width, img_height))

test_image_array = tf.keras.preprocessing.image.img_to_array(test_image)
test_image_array = tf.expand_dims(test_image_array, axis=0)
test_image_array /= 255.0

# Predict the class probabilities
predictions = loaded_model.predict(test_image_array)

# Get the predicted class label
predicted_class_index = tf.argmax(predictions, axis=1)[0]
predicted_class_name = class_mapping[int(predicted_class_index)]
print('Predicted class:', predicted_class_name)

1/1 [==============================] - 0s 58ms/step
Predicted class: Tracks and Trousers


In [21]:
# Save the model
model.save('image_classification_model.h5')
print('Model saved.')

# Load the model


Model saved.
Model loaded.
